<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/AkashNarayanan" target="_blank"> Akash Narayanan B</a></td>    
</table>
<hr>

Let us look at the different ways to convert between QUBO and BQM formulations.

# Converting a QUBO to a BQM

## QUBO - NumPy Representation

We have already learned to define a QUBO matrix as a NumPy matrix. It is possible to convert a NumPy matrix to a Binary Quadratic Model by using the `from_numpy_matrix` method.

Let's consider the following objective function

$$f(x_1, x_2, x_3, x_4) = - 5x_1 - 3x_2 - 8x_3 - 6x_4 + 4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$$

The QUBO matrix Q for the objective function is 

$$
Q = \begin{bmatrix}
        -5  &  4   &  8   &  0  \\ 
        0   &  -3  &  2   &  0  \\ 
        0   &  0   &  -8  &  10  \\ 
        0   &  0   &  0   &  -6  \\ 
    \end{bmatrix}
$$

The NumPy matrix `Q` is

In [1]:
import numpy as np

Q = np.array([[ -5,  4,  8,  0],
              [  0, -3,  2,  0],
              [  0,  0, -8, 10],
              [  0,  0,  0, -6]])

## `from_numpy_matrix` Method

### Parameters

- `Q` - The QUBO as a NumPy matrix
- `offset` (optional) - Constant offset

Now let's create a BQM from the above NumPy matrix. We have to pass `Q` as an argument to the `from_numpy_matrix()` method of the `BinaryQuadraticModel` class.

In [2]:
import dimod

bqm_np = dimod.BinaryQuadraticModel.from_numpy_matrix(Q)

print(bqm_np)

BinaryQuadraticModel({0: -5.0, 1: -3.0, 2: -8.0, 3: -6.0}, {(0, 1): 4, (0, 2): 8, (1, 2): 2, (2, 3): 10}, 0.0, 'BINARY')


So far we have used strings as keys to represent the variables in linear and quadratic arguments of a BQM class. It is also valid to use integers as keys. In the above output, the integers represent the position of the values in the matrix.

For example, the first term of the linear part `0: -5.0` represents that the value `-5.0` is at the position `(0, 0)` in the matrix. The first term of the quadratic part `(0, 1): 4` represents that the value `4` is at the position `(0, 1)` in the matrix.

<div class="alert alert-block alert-info">
The keys for the linear and quadratic arguments of the BQM class can either be strings or integers.
</div>

## QUBO - Dictionary Representation

We can also represent a QUBO problem as a dictionary. What is the need for it you may ask? Dictionary representation can be very helpful for problems with a large number of variables.

In the dictionary representation, only the non-zero terms of a QUBO matrix are considered. This saves up space and improves the efficiency of the problem solving process.

Let's consider a $3 \times 3$ matrix

$$
Q = \begin{bmatrix}
        Q_{11} & Q_{12} & Q_{13}  \\ 
        0      & Q_{22} & Q_{23}  \\ 
        0      & 0      & Q_{33}  \\ 
    \end{bmatrix}
$$

In the dictionary representation, the keys should be variables and their values should be the coefficients associated with these variables. The variables can be represented either as a tuple of variable names or as a tuple of numbers. The key for the term $Q_{11}$ in the above matrix can be represented as

- `('x1', 'x1')` - Tuple of variable names
- `(0, 0)` - Tuple of numbers that indicate the position of the term in the matrix

The advantage of dictionary representation becomes apparent when we consider a large QUBO matrix.

<div class="alert alert-block alert-danger">
This example has to be improved.
</div>

$$
Q_L = \begin{bmatrix}
        \bf{3} & 0 & 0 & 0 & 0 & \bf{4} & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & \bf{9} & 0 \\
        0 & 0 & 0 & \bf{1} & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & \bf{4} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \bf{8} \\
      \end{bmatrix}
$$

The above matrix $Q_L$ can be represented as

In [3]:
Q_Large = {('x1', 'x1'): 3, ('x4', 'x4'): 1, ('x5', 'x5'): 4,
           ('x8', 'x8'): 8, ('x1', 'x6'): 4, ('x3', 'x7'): 9}

How simple and concise was that?!

A QUBO problem in dictionary form can be converted to a BQM using the `from_qubo` method of the `BinaryQuadraticModel` class.

## `from_qubo` Method

### Parameters

- `Q` - The QUBO in dictionary form
- `offset` (optional) - Constant offset

Now let's a create a BQM by passing the dictionary `Q_Large` as an argument to the `from_qubo` method.

In [4]:
bqm_qubo = dimod.BinaryQuadraticModel.from_qubo(Q_Large)

print(bqm_qubo)

BinaryQuadraticModel({x1: 3.0, x4: 1.0, x5: 4.0, x8: 8.0, x6: 0.0, x3: 0.0, x7: 0.0}, {('x1', 'x6'): 4, ('x3', 'x7'): 9}, 0, 'BINARY')


# Converting a BQM to a QUBO

It is also possible to convert a Binary Quadratic Model to a QUBO model.

## `to_numpy_matrix` Method

### Parameter

- `variable_order` (optional) - Variable order as a list should be passed as an argument if there are variable names as keys in the BQM.

Let us consider the following BQM where the keys for the `linear` and `quadratic` arguments are variable names as strings.

In [5]:
bqm_str = dimod.BinaryQuadraticModel({'x1': -5.0, 'x2': -3.0,
                                      'x3': -8.0, 'x4': -6.0},
                                     {('x1', 'x2'): 4, ('x1', 'x3'): 8,
                                     ('x2', 'x3'): 2, ('x3', 'x4'): 10},
                                     0,
                                     'BINARY')

In this case, the variable order as a list should be passed as an argument to the `to_numpy_matrix` method. Without that there would be an error.

In [6]:
np_mat_str = bqm_str.to_numpy_matrix(['x1', 'x2', 'x3', 'x4'])

print(np_mat_str)

[[-5.0 4 8 0.0]
 [0 -3.0 2 0.0]
 [0 0 -8.0 10]
 [0 0 0 -6.0]]


If the keys of the `linear` and `quadratic` arguments are numbers, the `variable_order` is optional.

In [7]:
bqm_num = dimod.BinaryQuadraticModel({0: -5.0, 1: -3.0, 2: -8.0, 3: -6.0},
                                     {(0, 1): 4, (0, 2): 8,
                                      (1, 2): 2, (2, 3): 10},
                                     0.0,
                                     'BINARY')

np_mat_num = bqm_num.to_numpy_matrix()

print(np_mat_num)

[[-5.0 4 8 0.0]
 [0 -3.0 2 0.0]
 [0 0 -8.0 10]
 [0 0 0 -6.0]]


## `to_qubo` Method

This method can be used to convert a Binary Quadratic Model to a QUBO form. If the `vartype` of the BQM is `'SPIN'`, it is converted to `'BINARY'`.

This method returns a tuple of form `(biases, offset)` where `biases` is a dictionary of the linear and quadratic terms and `offset` is a number.

Let's consider the same `bqm_str` used in the previous example. The QUBO form of the BQM is

In [8]:
qubo_str = bqm_str.to_qubo()

print(qubo_str)

({('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10, ('x1', 'x1'): -5.0, ('x2', 'x2'): -3.0, ('x3', 'x3'): -8.0, ('x4', 'x4'): -6.0}, 0)


In the above output,

- The first terms of the tuple correspond to the linear and quadratic terms of the QUBO
    
    ```python
    {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10,
     ('x1', 'x1'): -5.0, ('x2', 'x2'): -3.0, ('x3', 'x3'): -8.0, ('x4', 'x4'): -6.0}
    ```
    
- The second term corresponds to the offset `0`.    